In [ ]:
import pandas as pd
pd.options.display.float_format = "{:.2f}".format

In [ ]:
#csv파일을 불러온다
rock_samples = pd.read_csv("C:/Users/EL11/dataschool/rocksamples.csv") #이과정에서 오류가 났는데 경로 복사할때 '\'로 들어가서 수기로 '/'로 고쳐줌

In [ ]:
#데이터프레임 살펴보기 1. 위에서 부터 5개
rock_samples.head()

In [ ]:
#데이터프레임 살펴보기 1. 밑에서 부터 5개
rock_samples.tail()

In [ ]:
rock_samples.info()
#Column 아래로 INDEX
#Dtype은 데이터 타입관련, 가장 아래 dytypes는 각 컬럼의 데이터 타입
#(total 6 columns)_전체 행수
#Memory usage 메모리 사용량

In [ ]:
rock_samples.describe()

In [ ]:
#누락값 검사
rock_samples.isnull().sum()

#subType에 누락값이 3개 있는 것을 확인

In [ ]:
rock_samples.isnull()

열단위 데이터 추출

In [ ]:
sample_series = rock_samples['Weight (g)']
sample_series

In [ ]:
sample_df = rock_samples[['Type','Weight (g)']]
sample_df

In [ ]:
rock_samples['Weight (kg)'] = rock_samples['Weight (g)'] / 1000

람다lambda 함수도 사용가능하다

단위가 g에서 kg로 바뀐것을 알 수 있다.

In [ ]:
rock_samples.head()

In [ ]:
missions = pd.DataFrame() #빈데이터 프레임을 만들어 missions 라는 변수에 할당
missions

In [ ]:
type(missions)

In [ ]:
type(rock_samples['Mission'])

In [ ]:
rock_samples['Mission'].unique() #Series.unique()는 시리즈에서 고유한 값들을 찾아주는 명령어

In [ ]:
missions['Mission'] = rock_samples['Mission'].unique()
missions

In [ ]:
missions.info()

In [ ]:

#groupby 컬럼의 고유값에 따라 묶어서 집계 또는 통계 처리를 할 때 그룹바이 명령을 적용
sample_total_weight = rock_samples.groupby('Mission')['Weight (kg)'].sum()
sample_total_weight

In [ ]:
type(sample_total_weight)

In [ ]:
#pd.merge : 데이터프레임의 컬럼명 중에서 시리즈의 인덱스명과 같은 컬럼을 기준으로 데이터프레임과 시리즈를 병합
#pd.merge( df, s, on=‘시리즈 인덱스와 같은 데이터프레임의 컬럼명’ )
missions = pd.merge(missions, sample_total_weight, on = 'Mission')

In [ ]:
missions

In [ ]:
#missions.rename( columns={ 변경전 컬럼명 : 변경후 컬럼명 }, inplace=True )
missions.rename(columns={'Weight (kg)' : 'Sample Weight (kg)'}, inplace=True)
missions

In [ ]:
missions['Weight diff'] = missions['Sample Weight (kg)'].diff()
missions #첫번째는 이전과 차이가 없기 때문에 NaN

In [ ]:
missions.fillna(value=0,inplace=True) #NaN을 0으로 바꿔주는 작업
missions

In [ ]:
#달탐사선의 중량 데이터를 추가한다
missions['Lunar module (LM)'] = ['Eagel (LM-5)','Interpid (LM-6)','Antares (LM-8)','Falcon (LM-10)','Orion (LM-11)','Challenger (LM-12)']
missions['LM mass (kg)'] = [15103,15235,15264,16430,16445,16456] #입력
missions['LM mass diff'] = missions['LM mass (kg)'].diff() #이전값과의 차이
missions['LM mass diff'] = missions['LM mass diff'].fillna(value=0)

In [ ]:
missions

In [ ]:
#명령모듈의 추가
missions['Command Module (CM)'] = ['Columbia (CM-107)','Yankee Clipper (CM-108)','Kitty Hwak (CM-110)','Endeavor (CM-110)','Casper (CM-113)','America (CM-114)']
missions['CM mass (kg)'] = [5560,5609,5758,5875,5840,5960]
missions['CM mass diff'] = missions['CM mass (kg)'].diff()
missions['CM mass diff'] = missions['CM mass diff'].fillna(value=0)
missions

In [ ]:
missions['Total Weight (kg)'] = missions['LM mass (kg)'] + missions['CM mass (kg)'] #무게의 합
missions['Total Weight diff'] = missions['LM mass diff'] + missions['CM mass diff'] #중량차의 합
missions

In [ ]:
#로켓안의 실리는 하중 Payload Sample-to-weight ratio
saturnVPayload = 43500
#승무원 영역이 차지하는 비율
missions['Crewed area : Payload'] = missions['Total Weight (kg)'] / saturnVPayload

In [ ]:
#승무원 영역의 샘플중량 비율
missions['Sample : Crewed area'] = missions['Sample Weight (kg)'] / missions['Total Weight (kg)']

In [ ]:
#페이로드에 대한 샘플 중량 비율
missions['Sample : Payload'] = missions['Sample Weight (kg)'] / saturnVPayload

In [ ]:
missions

In [ ]:
#아르테미스 임무 수행시 예상값 측정
artemis_crewedArea = 26520
artemis_mission = pd.DataFrame({'Mission':['artemis1','artemis1b','artemis2'],'Total Weight (kg)':[artemis_crewedArea,artemis_crewedArea,artemis_crewedArea],'Payload (kg)':[26988,37965,42955]})
artemis_mission

In [ ]:
missions['Crewed area : Payload'].min() #최소값

In [ ]:
missions['Crewed area : Payload'].max() #최대값

In [ ]:
missions['Crewed area : Payload'].mean() #평균값

In [ ]:
missions['Crewed area : Payload'].median() #중위값

In [ ]:
missions

In [ ]:
missions['Sample Weight (kg)'].sum() #아폴로 임무에서 구해온 암석샘플 중량의 총합

In [ ]:
crewedArea_payload_ratio = missions['Crewed area : Payload'].mean()
sample_crewedArea_ratio =  missions['Sample : Crewed area'].mean()
sample_payload_ratio = missions['Sample : Payload'].mean()
print(crewedArea_payload_ratio)
print(sample_crewedArea_ratio)
print(sample_payload_ratio)

In [ ]:
artemis_mission['Sample weight from total (kg)'] = artemis_mission['Total Weight (kg)']*sample_crewedArea_ratio
artemis_mission['Sample weight from payload (kg)'] = artemis_mission['Payload (kg)']*sample_payload_ratio
artemis_mission

In [ ]:
#아르테미스 샘플 예상 중량은 이렇게
artemis_mission['Estimated sample weight (kg)'] = (artemis_mission['Sample weight from payload (kg)'] + artemis_mission['Sample weight from total (kg)'])/2
artemis_mission

In [ ]:
rock_samples.head()

In [ ]:
rock_samples['Remaining (kg)'] = rock_samples['Weight (kg)']*(rock_samples['Pristine (%)']*0.01)
rock_samples.head()

In [ ]:
rock_samples.describe()

In [ ]:
rock_samples['Weight (kg)'] >= .16

In [ ]:
rock_samples['Pristine (%)'] <= 50

In [ ]:
(rock_samples['Weight (kg)']>=.16)&((rock_samples['Pristine (%)']<=50))

In [ ]:
low_samples = rock_samples.loc[(rock_samples['Weight (kg)']>=.16)&(rock_samples['Pristine (%)']<=50)]
low_samples.head()

In [ ]:
low_samples.groupby('Type')['Weight (kg)'].count()

In [ ]:
low_samples['Type'].isin(['Basalt','Breccia'])

In [ ]:
low_samples.loc[low_samples['Type'].isin(['Basalt','Breccia'])]

In [ ]:
needed_samples = low_samples.loc[low_samples['Type'].isin(['Basalt','Breccia'])]
needed_samples.head()

In [ ]:
needed_samples.info()

In [ ]:
needed_samples.groupby('Type')['Weight (kg)'].sum()

In [ ]:
#위 결과로 basalt, breccia는 수집이 많았지만 사용량도 많아 추가 수집 필요
rock_samples.groupby('Type')['Weight (kg)'].sum()

In [ ]:
#위 결과로 crustal 과 special 항목이 엄청 적은걸 볼수있다.
needed_samples.head()

In [ ]:
rock_samples.loc[rock_samples['Type']=='Crustal']

In [ ]:
needed_samples = pd.concat([needed_samples, rock_samples.loc[rock_samples['Type'] == 'Crustal']]) 
needed_samples.info()

In [ ]:
needed_samples_overview = pd.DataFrame()
needed_samples_overview

In [ ]:
type(needed_samples_overview)

In [ ]:
needed_samples['Type'].unique()

In [ ]:
needed_samples_overview['Type'] = needed_samples['Type'].unique() #중복 없는 암석 유형 추출
needed_samples_overview

In [ ]:
needed_samples.groupby('Type')['Weight (kg)'].sum()

In [ ]:
type(needed_samples.groupby('Type')['Weight (kg)'].sum())

In [ ]:
needed_samples.groupby('Type')['Weight (kg)'].sum().reset_index()

In [ ]:
type(needed_samples.groupby('Type')['Weight (kg)'].sum().reset_index())

In [ ]:
needed_samples_overview

In [ ]:
needed_samples_weights = needed_samples.groupby('Type')['Weight (kg)'].sum().reset_index()
needed_samples_weights

In [ ]:
needed_samples_overview = pd.merge(needed_samples_overview, needed_samples_weights, on = 'Type') #df합치기
needed_samples_overview

In [ ]:
needed_samples_overview.rename(columns={'Weight (kg)': 'Total Weight (kg)'},inplace=True)
needed_samples_overview

In [ ]:
needed_samples.groupby('Type')['Weight (kg)'].mean() #암석중량 평균

In [ ]:
needed_samples_ave_weight = needed_samples.groupby('Type')['Weight (kg)'].mean().reset_index() #데이터프레임으로변환
needed_samples_ave_weight

In [ ]:
needed_samples_overview = pd.merge(needed_samples_overview, needed_samples_ave_weight, on = 'Type')
needed_samples_overview

In [ ]:
needed_samples_overview.rename(columns={'Weight (kg)':'Average Weight (kg)'}, inplace=True) #컬럼명변경
needed_samples_overview

In [ ]:
total_rock_count = rock_samples.groupby('Type')['ID'].count().reset_index()
total_rock_count

In [ ]:
needed_samples_overview = pd.merge(needed_samples_overview, total_rock_count, on = 'Type')
needed_samples_overview.rename(columns={'ID':'Numbers of samples'}, inplace=True)
needed_samples_overview

In [ ]:
total_rocks = needed_samples_overview['Numbers of samples'].sum()
total_rocks

In [ ]:
needed_samples_overview['Percentage of rocks'] = needed_samples_overview['Numbers of samples'] / total_rocks
needed_samples_overview

In [ ]:
artemis_mission

In [ ]:
artemis_ave_weight = artemis_mission['Estimated sample weight (kg)'].mean()
artemis_ave_weight

In [ ]:
needed_samples_overview['Weight to collect'] = needed_samples_overview['Percentage of rocks']*artemis_ave_weight
needed_samples_overview['Rocks to collect'] = needed_samples_overview['Weight to collect']/needed_samples_overview['Average Weight (kg)']
needed_samples_overview

In [ ]:
import matplotlib.pyplot as plt

missions = rock_samples.groupby("Mission")["Weight (kg)"].sum().reset_index()

plt.figure(figsize=(8,5))
plt.bar(missions["Mission"], missions["Weight (kg)"], color="skyblue")
plt.title("Total Sample Weight by Mission")
plt.xlabel("Mission")
plt.ylabel("Weight (kg)")
plt.xticks(rotation=45)
plt.tight_layout()
plt.savefig("mission_total_weight.png")  # 이미지 저장
plt.show()


In [ ]:
types = rock_samples.groupby("Type")["Weight (kg)"].sum()

plt.figure(figsize=(6,6))
plt.pie(types, labels=types.index, autopct="%.1f%%", startangle=90)
plt.title("Rock Types Proportion")
plt.savefig("rock_type_ratio.png")
plt.show()


In [ ]:
plt.figure(figsize=(8,5))
rock_samples.boxplot(column="Remaining (kg)", by="Mission", grid=False)
plt.title("Remaining (kg) by Mission")
plt.suptitle("")  # 상단 중복 타이틀 제거
plt.xlabel("Mission")
plt.ylabel("Remaining (kg)")
plt.tight_layout()
plt.savefig("remaining_by_mission.png")
plt.show()
